# **Extração de emoções e Análise de Sentimentos em Letras de músicas do Forró**

In [1]:
import pandas as pd

In [72]:
#!pip install -r requirements.txt

In [90]:
df = pd.read_csv('./lyrics.csv', sep=';', encoding="utf-8")
df.shape

(13405, 7)

# Preprocessing

1. Remoção dos títulos das músicas que contenham os caracteres '()' 

In [91]:
df['title'].replace("\((.*?)\)", '', regex=True, inplace=True)



2. Remoção das músicas duplicadas a partir do título

In [92]:
df['title'] = df['title'].str.lower()
df['title'] = df['title'].str.strip()

titles = [i for i in df['title']]

duplicates = [i for i in titles if titles.count(i) > 1]
print(f'Quantidade de duplicatas: {len(duplicates)}')

Quantidade de duplicatas: 3484


In [94]:
df = df[~df['title'].isin(duplicates)]
print(f'Quantidade de músicas após remoção das duplicatas: {df.id.count()}')

Quantidade de músicas após remoção das duplicatas: 9921


3. Remoção das músicas sem data

In [95]:
df = df[df['year'] > 0]

print(f'Quantidade de músicas com data: {df.id.count()}')

Quantidade de músicas com data: 8176


4. Remoção dos registros sem letras

In [96]:
df = df[~df['lyric'].isnull()]
print(f'Quantidade de registros com letras: {df.lyric.count()}')

Quantidade de registros com letras: 8133


### Carregando dataframes

In [15]:
"""
Lexicon source is (C) 2016 National Research Council Canada (NRC) 
and library is for research purposes only.  
Source: http://sentiment.nrc.ca/lexicons-for-research/
"""
lexicon = pd.read_csv('./pt-nrc-lexicon.csv', sep=';', encoding='utf-8')

In [55]:
#gerando decadas
df['decade'] = df['year'].apply(decade)
#df.tail()

,id,title,artist,composer,album,year,lyric,clean_lyric,unique_words,decade
8128,13398,probleminha,DJ Ivis,DJ Ivis / Junior Gomes.,Eu Ouvi Brasil,2021,Sabe porque não saio da tua mente? É que na ca...,Sabe porque não saio da tua mente? É que na ca...,"{tu, largar, Que, logo, Assume, ele, quer, Tu,...",20
8129,13399,me chama vai,DJ Ivis,Val Lima.,Eu Ouvi Brasil,2021,Tá faltando o quê? Pra tu ligar e eu atender A...,Tá faltando o quê? Pra tu ligar e eu atender A...,"{tu, Cuida, ligar, hoje, amor, já, fazer, vai,...",20
8130,13400,bundão,DJ Ivis,NaN,Eu Ouvi Brasil,2021,Avisa que ela voltou Tá online na parada Ficou...,Avisa que ela voltou Tá online na parada Ficou...,"{mandando, hoje, Que, fogo, ex, parada, Danny,...",20
8131,13403,seu love sou eu,DJ Ivis,Sammy Coelho / Valter Danadão.,Eu Ouvi Brasil,2021,Vai começar de novo Vai quebrar a cara Depois ...,Vai começar de novo Vai quebrar a cara Depois ...,"{tu, Você, nunca, iludir, Que, cena, Achando, ...",20
8132,13404,amor é o que eu sinto,O Melhor do Forró Gospel,Sebhasttião Alves.,Forró Gospel Mais Tocadas,2022,Teu olhar é tão belo Teu sorriso é tão lindo M...,Teu olhar é tão belo Teu sorriso é tão lindo M...,"{Teu, sorriso, lindo, por, não, és, o, tão, Am...",20


In [57]:
#removendo ruídos das letras
df['clean_lyric'] = df['lyric'].apply(clean_lyrics)

In [58]:
#removendo letras nulas
df['clean_lyric'] = df[~df['clean_lyric'].isnull()]['clean_lyric']

In [59]:
#armazenando
df.to_csv('clean_lyrics.csv', index=False)

In [52]:
#função para criar decadas
decade = lambda year: int(((((year - (year % 10)) / 10) % 10) * 10))

In [62]:
df = pd.read_csv('./clean_lyrics.csv')
df.head()

,id,title,artist,composer,album,year,lyric,clean_lyric,unique_words,decade
0,2360,joão teimoso,Marília Batista,Noel Rosa / Marilia Batista.,Poeta da Vila,1953,Tenho mais o que fazer Não discuto com teimoso...,discuto teimoso perder precioso viver joão tei...,"{'Este', 'Não', 'viver', 'fome', 'em', 'nome',...",50
1,2361,morena sereia,Marília Batista,José Maria De Abreu 1936 / Noel Rosa.,Poeta da Vila,1953,Morena sereia Que à beira-mar não passeia Que ...,morena sereia beiramar passeia senta praia dei...,"{'dia', 'tu', 'esperança', 'Que', 'castelos', ...",50
2,2362,"parabéns, guanabara",Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,"Rio, querida Guanabara Eu sou gente também Ace...",rio querida guanabara gente aceito parabéns pa...,"{'cantando', 'querida', 'dos', 'calcanhar', 'D...",50
3,2363,o bom xaxador,Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,Depois do baião foi que veio o forró Depois do...,baião veio forró forró veio rojão pisada chão ...,"{'pisada', 'Não', 'baião', 'Depois', 'xaxador'...",50
4,2364,cheguei agora,Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,Beira mar beira do rio Ê ê beira mar Cheguei a...,beira mar beira rio ê ê beira mar cheguei beir...,"{'Tem', 'candomblé', 'canta', 'nunca', 'Chegue...",50


In [63]:
# gerando colunas de palavras únicas
df['unique_words'] = df['clean_lyric'].apply(generate_unique)
df.head()

nan


AttributeError: 'float' object has no attribute 'split'

In [44]:
df[df['clean_lyric'].isnull()]

,id,title,artist,composer,album,year,lyric,clean_lyric,unique_words
1025,3658,um casal de advogados,Trio Nordestino,NaN,Trio Nordestino e Acompanhamento,1963,[ Eu conheci um casal de advogadosQue trabalha...,NaN,"{Fez, entrarE, profissãoE, apelarEntrou, cara,..."
1029,3663,"dá, dá, dá nega",Trio Nordestino,NaN,Trio Nordestino e Acompanhamento,1963,[ Quando ela ri faz um buraco na bochechaEu qu...,NaN,"{ta, ele, quero, me, fofinho, o, tão, dá, [, b..."
6271,10902,deusa de itamaracá,Rasta Chinela,NaN,Rasta Chinela,2003,"[ Sei que é para me matarTeu olhar, teu olharL...",NaN,"{vouPra, amor, vezes, possa, desejarDeusa, mat..."
6282,10921,eu duvido,Rasta Chinela,NaN,Rasta Chinela,2003,[ Você pode revirar o mundoQue não vai achar n...,NaN,"{ninguém, Você, amorEu, cena, ta, terrorSinto,..."
7123,12131,mão na urtiga,Zaíra,NaN,O Forró de Dona Zaíra,2008,(Instrumental - part. Hermeto Paschoal),NaN,"{-, (Instrumental, part., Paschoal), Hermeto}"


In [46]:
df.loc[1025, 'lyric']

'[ Eu conheci um casal de advogadosQue trabalham separados na mesma profissãoE no processo que o doutor foi defenderA doutora foi fazer uma tremenda acusaçãoEla chegou e entrou na primeira varaFoi botando cara a cara a testemunha ocularE no final do processo ela perdeuE num instante recorreuE noutra vara foi entrarE dava pena ver o réu com aquela caraSai de vara entra em vara querendo absolvição E toda vez que o juiz absolviaNoutra vara ela ia querendo a condenaçãoUm certo dia ela acusou o moço Fez um grande alvoroço condenando o infelizE proibiu todo dia usar recurso De tentar mudar o curso de vara e de juizDisse ao doutor que ele quebraria a caraSe tentasse entrar na vara com a intenção de apelarEntrou na avara com o juizo ela estava E o processo segurava em toda vara do lugar ]'

In [ ]:
#normalizando palavras
df['clean_lyric'] = df['clean_lyric'].apply(normalize_text)
df.to_csv('clean_lyrics.csv', index=False)

### Funções utilitárias

In [36]:
def generate_unique(text):
    if type(text) is not str:
        print(text)
    return set(text.split())

In [3]:
#carregando lista de acordes para remoção dentro das letras
chords_df = pd.read_csv('./chords.txt')
chords_dicty = chords_df.to_dict()
chords = set()

for i in chords_dicty['chord']:
    chords.add(chords_dicty['chord'][i].strip())
    
#função para remoção de acordes dentro de um texto
def remove_chords(text):
    txt_list = text.split()
    return (' ').join(
            word.strip()
            for word in txt_list
            if word.strip() not in chords
        )    

In [4]:
import string
import re
import spacy

nlp = spacy.load('pt_core_news_lg')

def clean_lyrics(text):
  txt = ''
  #removendo acordes
  txt = remove_chords(text)
  #removendo tags de repetição 2x, 4x, etc
  txt = txt.replace('[', '(').replace(']', ')')
  txt = re.sub(r'\([\d]x\)', '', txt)
  #removendo palavras de orientação musical
  txt = re.sub(r'\((.*?)\)', '', txt)  
  #removendo pontuacao
  txt = txt.translate(str.maketrans("", "", string.punctuation))
  #caracteres em minusculo   
  txt = txt.lower()
  #remove stop words
  doc = nlp(txt)
  txt = ' '.join(token.text 
                  for token in doc 
                  if not token.is_stop
                )
    
  return txt

In [31]:
from enelvo.normaliser import Normaliser

def normalize_text(text):
    norm = Normaliser(capitalize_pns=True)
    return norm.normalise(text)